# HW1: Developing an Information Retrieval System with Spelling Correction and Wildcard Queries

## About
This project aims to enhance the Information Retrieval (IR) system developed in the first assignment by handling
Spelling Correction and Wildcard Queries. This assignment can be completed independently of Project 1. You can
find the data here.

## importing libraries
Let's first import libraries we might need. we will use some libraries as typing checking, and some libraries like string just for punctuation.
and also, as it is said in Problem file, we can use nltk library for tokenizing. I implement two way for tokenizing, one using nltk and one without.

In [1]:
import os
import string
from typing import List

import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/amir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/amir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Main classes
Now let's define our classes. I define two main classes and two small for this project. Trie and Information Retrieval System are the two main classes and token and node are two small class that I use them in main classes respectively.
### Algorithm
Befor going to implementation, let's summarize our algorithm and approach:
First of all, we get a list of documents in format of string; we need to tokenize all these documents. we can split all words using whitespace. Then, we need to get rid of punctuation. This will be done using nltk. at last, it's better to lower case all words so we can index for example "another" and "Another" in same. After tokenizing, we have to create our posting_list, which is the core of this project. To do this, we should loop over all documents, then inside this loop, we loop over all the tokens that are in the current document. then we check if the length of posting_list is zero, then we add this token as first word. else if the length of posting_list is more than 0, we find the correct index of the token in posting_list alphabetically. then we check if this token, has been already in posting_list, we just add the current document index in tokens.docs, else, we add this token in the posting_list, then add the current document index. Also we store index of occurence of the token in the relevant documents.
After creating the posting list, we should add all words and their permutations to a Trie tree; We need to do this to handle wildcard queries.
this can be done by insert all words of the posting_list, to the trie. The algorithm is completely describe in docstring of the Trie class.
Now we create our posting list, we can easily search a query. To do this, I have a main search function that do one things at first, it check if each side of the query, contains wildcard or not. if yes, it replace that side by the list of a word that matches the wildcard query, and if it does not contain wildcard, it just replace that word by a list that contain just the same word; we do it according to the fact that in each case of (AND, OR, NOT, Near) we should union the list of each side of the main operation, and then do the main operation.
Now let's see what do we do in wildcard query; for each single token that contain *, If there is just one *, it will add a '$' at the end of the word, and then permutate it such that the * pose on the end of the string, and then search it in prefix_trie and will return the all match tokens. If there are two *, it convert two * and the characters between these two into one *. Then do the same thing for having one *. And from resulted matched, it search if they have the sequence characters of between two * in the input token.
Now we can do the boolean or aproxy queries. Before explain each, there is a note: in each operation, for each word we try to find it from the posting list, and if can't, we do the spell correction. Now let's define boolean and aproxy operations:
For just getting the documents of a one word, we have the function get_word_docs; this simple function gets a token and will return all index of documents that this token is appeared in. For intersecting(AND) 2 words, we just need to return document's index of our 2 words that are occurred in same documents. for union(OR), we gather index of word1 with word2. for NOT, we just need to return all document's index that are not in the list of documents that contain our word. And at last, for near, we just need to find documents that either each of these word has been occurred near by at most k words on left or right. we have to cases:
 * 1)the first_word occurred before second_word.
 * 2)second_word occurred before first_word.
so we loop over a tuple (p1, p2) to cover ordered occurrence of each word. then for each word we loop over its documents, and in each document, we loop over index of that the token is occurred. then if other word(let's say second) is in documents[word1_idx:word1_idx+length], we add the current document index.

### challenges
I had two challelnges for this project:
* 1)create the prefix tree and retrieve wildcard query. Soulution: add all circular permutation of words to the tree
* 2)spell checking of a word. Solution: compare all edit distance of a word with all posting list words
we will explain the algorithms in function too.

## Utility functions
Let's define a function to get all circular permutation of a word

In [2]:
def get_all_permutations(input_str: str):
    """
    this function will generate all circular permutation of a string
    """
    perm_list = []
    n = len(input_str)
    for i in range(n):
        permuterm = input_str[i:n] + input_str[0:i]
        perm_list.append(permuterm)
    return perm_list

def edit_distance(word1: str, word2: str):
    """
    this function will calculate the distance of between two word. The distance means the total number of operation
    that we need to convert each word to other
    :param word1: str
        first word you want to calculate distance of with another
    :param word2:
        second word you want to calculate distance of with another
    :return:
        the edit distance of between word1 and word2
    """
    dp = [[0 for j in range(len(word2))] for i in range(len(word1))]
    for i in range(1, len(word1)):
        dp[i][0] = i
    for j in range(1, len(word2)):
        dp[0][j] = j
    for i in range(1, len(word1)):
        for j in range(1, len(word2)):
            dp[i][j] = min(dp[i - 1][j - 1] + (0 if word1[i] == word2[j] else 1),
                           dp[i - 1][j] + 1,
                           dp[i][j - 1] + 1)
    return dp[-1][-1]


## Preprocessing
Let's define our preprocess functions

In [3]:
import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Initialize NLTK stop words
stop_words = set(stopwords.words('english'))


def read_documents(doc_folder):
    """
    Reads all text files in the specified directory and returns a dictionary
    with document ids as keys and the content as values.
    """
    documents = {}
    for doc_file in os.listdir(doc_folder):
        if doc_file.endswith('.txt'):
            doc_path = os.path.join(doc_folder, doc_file)
            with open(doc_path, 'r', encoding='utf-8') as file:
                documents[doc_file] = file.read()
    return documents


def preprocess_text(text):
    """
    Preprocesses the text by tokenizing, converting to lowercase, removing punctuation,
    and filtering out stop words.
    """
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens


def preprocess_documents(documents):
    """
    Apply text preprocessing to each document in the dictionary.
    """
    preprocessed_docs = []
    for doc_id, content in enumerate(documents):
        preprocessed_docs.append(preprocess_text(content))
    return preprocessed_docs


## Indexing
Now let's define our main class:

In [4]:
import nltk
import string
from typing import List

from src.utils import edit_distance


class Token:
    def __init__(self, word: str):
        self.word = word
        self.docs = []

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word


class InvertedIndex:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    documents: List
        list of documents in format of string.
    posting_list: List[Token]
        list of Token objects. Tokens store a string and document's indexes
    stop_word: set
        set of stop words to check when tokenizing
    case_sensitive: bool
        a boolean to determine whether we want to distinguish between lowercase and uppercase form.

    Methods
    -------
    Methods defined here:
        __init__(self, documents: List, case_sensitive=False):
            Constructor will set initial attributes like documents and case_sensitive. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then chagnes to list of lists of strings
            :return
                None
        add_document(self, doc_idx, doc):
            this function will add a document to the posting list. it will loop over all tokens in the document and
            add them to the posting list.
            :param doc_idx:
                index of the document in the documents list
            :param doc:
                list of tokens in the document
            :return:
                None
        create_posting_list(self):
            calling this function, will create posting list of all occurred words cross all documents.
            :parameter
                None
            :return
                None
        get_token_index(self, x):
            this function find index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        get_token(self, token):
                This function will return the token object that contains docs informations. if the given token is not in the
                posting_list, it return the spell corrected token.
                :param token:
                    token you want to fetch it from posting list
                :return:
                    return the instance of token from posting list
    """

    def __init__(self, documents: List, case_sensitive=False):
        """
        Constructor will set initial attributes like documents and case_sensitive. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then chagnes to list of lists of strings
            :return
                None
        """
        self.documents = documents
        self.posting_list: List[Token] = []
        self.stop_words: set = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))
        self.case_sensitive = case_sensitive

    def add_document(self, doc_idx, doc):
        for token_idx, token in enumerate(doc):
            if len(self.posting_list) == 0:
                self.posting_list.append(Token(token))
                self.posting_list[0].docs.append({'doc_idx': doc_idx, 'indexes': [token_idx]})
                continue
            i = 0
            while i < len(self.posting_list) and token > self.posting_list[i].word:
                i += 1
            if i == len(self.posting_list):
                self.posting_list.append(Token(token))
                # self.posting_list[i].post_idx.append(post_idx)
            elif token != self.posting_list[i].word:
                self.posting_list.insert(i, Token(token))

            if doc_idx not in [elem['doc_idx'] for elem in self.posting_list[i].docs]:
                self.posting_list[i].docs.append({'doc_idx': doc_idx, 'indexes': [token_idx]})
            else:
                self.posting_list[i].docs[-1]['indexes'].append(token_idx)

    def create_posting_list(self):
        """
        calling this function, will create posting list of all occurred words cross all documents. in this function, we
        loop over all documents, then inside this loop, we loop over all the tokens that are in the current document.
        the we check if the length of posting_list is zero, then we add this token as first word. else if the length of
        posting_list is more than 0, we find the correct index of the token in posting_list alphabetically. then we check
        if this token, has been already in posting_list, we just add the current document index in tokens.docs, else, we
        add this token in the posting_list, then add the current document index.
            :parameter
                None
            :return
                None
        :return:
        """
        for doc_idx, doc in enumerate(self.documents):
            self.add_document(doc_idx=doc_idx, doc=doc)

    def spell_correction(self, word):
        """
        this function will return the nearest word from posting_list to a given word which is might be incorrect.
        :param word:
            the word you want to return correction of it from posting list
        :return:
            index of the nearest token from posting list to the given word
        """
        nearest_idx = 0
        nearest_val = edit_distance(self.posting_list[nearest_idx].word, word)
        for idx in range(len(self.posting_list)):
            temp_dist = edit_distance(self.posting_list[idx].word, word)
            if temp_dist < nearest_val:
                nearest_idx = idx
                nearest_val = temp_dist
        return nearest_idx

    def get_token_index(self, x):
        """
        this function find index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        """
        low = 0
        high = len(self.posting_list) - 1
        mid = 0
        while low <= high:
            mid = (high + low) // 2
            if self.posting_list[mid].word < x:
                low = mid + 1
            elif self.posting_list[mid].word > x:
                high = mid - 1
            else:
                return mid
        return -1

    def get_token(self, token):
        """
        This function will return the token object that contains docs informations. if the given token is not in the
        posting_list, it return the spell corrected token.
        :param token:
            token you want to fetch it from posting list
        :return:
            return the instance of token from posting list
        """
        p = self.get_token_index(token)
        if p == -1:
            p = self.spell_correction(token)
        return self.posting_list[p]


## Trie (prefixtree)
Let's define our prefix tree class

In [5]:
class TrieNode:
    """
    This class is the main class of nodes that are used in Trie class
    ...
    Attributes:
    ----------
    char: str
        character that is in node
    is_end: bool
        this attribute determines if the current node is the end of a word or not.
    children: dict
        this attribute contains the children nodes of a node.

    Methods:
    -------
    Methods defined here:
        __init__(self, char):
            Constructor will set initial attribute, char.
            :parameter
            ---------
            char:str
                the character which is in current node.
            :return
                None
        __repr__(self):
            represents the instance of node as the char attribute.
        __str__(self):
            print the instance of node as the char attribute.
    """

    def __init__(self, char):
        self.char = char
        self.is_end = False
        self.children = {}

    def __repr__(self):
        return self.char

    def __str__(self):
        return self.char


class Trie(object):
    """
    This class is the prefix tree, Or Trie. This tree contains nodes that each is a character and have children as the
    next character. In this tree, words are the nodes that their is_end attributes are True.
    ...
    Attributes:
    ----------
    root: TrieNode
        Root of the whole tree.

    Methods:
    -------
    Methods defined here:
        __init__(self):
            Constructor will create the root by TrieNode object.
            :parameter
                None
            :return
                None
        insert(self, word):
            This function insert a word into the tree. It first sets the current node to root of the tree,
            then loop over the characters of the word, and if the current char exists in children of the current node,
            sets the current node to that child, and else, it create a new_node by the current char and add it to the
            children of the current node. And after the loop, the is_end attribute of the last node will set to True.
            :parameter
                - word:str
                    the word you want to insert into the tree.
            :return
                The last node which is the end of the word.
        _dfs(self, node, prefix, result=[]):
            This function get a node(root of a tree or subtree) and a prefix, then return all words that contain the
            given prefix in the result argument.

            :parameter
                - node:TrieNode
                    the node to start with
                - prefix:str
                    the current prefix, for tracing a
                    word while traversing the trie
                - result:list
                    the list that the function will add all words into it.
            :return
                return all words that contain the given prefix in the given tree.

        dfs_helper(self, node, prefix):
            this function is a helper function for _dfs, because the _dfs function store results in its parameter.
            :parameter
                - node:TrieNode
                    the node to start with
                - prefix:str
                    the current prefix, for tracing a
                    word while traversing the trie
            :return
                return all words that contain the given prefix in the given tree.

        print_all_words(self):
            this function will print all words of a Trie.
            :parameter
                None
            :return
                None
        query(self, x:str):
            This function is as same as the dfs_helper function, but first check if the given word is in trie or not, then
            call the the dfs_helper with the root node and return all words contain x as prefix(include x if there exist).
            :parameter
                - x:str
                    the given word or prefix
            :return
                return all words contain x as prefix(include x if there exist).
    """

    def __init__(self):
        """
        The trie has at least the root node.
        The root node does not store any character
        """
        self.root = TrieNode("")

    def insert(self, word):
        """
        This function insert a word into the tree. It first sets the current node to root of the tree,
        then loop over the characters of the word, and if the current char exists in children of the current node,
        sets the current node to that child, and else, it create a new_node by the current char and add it to the
        children of the current node. And after the loop, the is_end attribute of the last node will set to True.
            :parameter
                - word:str
                    the word you want to insert into the tree.
            :return
                The last node which is the end of the word.
        """
        node = self.root
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node
        node.is_end = True
        return node

    def _dfs(self, node, prefix, result=[]):
        """
        This function get a node(root of a tree or subtree) and a prefix, then return all words that contain the
        given prefix in the result argument.
            :parameter
                - node:TrieNode
                    the node to start with
                - prefix:str
                    the current prefix, for tracing a
                    word while traversing the trie
                - result:list
                    the list that the function will add all words into it.
            :return
                 return all words that contain the given prefix in the given tree.
        """
        if node.is_end:
            # result.append(node)
            result.append(prefix + node.char)

        for child in node.children.values():
            self._dfs(child, prefix + node.char, result)

    def dfs_helper(self, node, prefix):
        """
        this function is a helper function for _dfs, because the _dfs function store results in its parameter.
            :parameter
                - node:TrieNode
                    the node to start with
                - prefix:str
                    the current prefix, for tracing a
                    word while traversing the trie
            :return
                return all words that contain the given prefix in the given tree.
        """
        result = []
        self._dfs(node, prefix, result=result)
        return result

    def print_all_words(self):
        """
        print_all_words(self):
            this function will print all words of a Trie.
            :parameter
                None
            :return
                None
        """
        for word in sorted(self.dfs_helper(self.root, '')):
            print(word)

    def query(self, x: str):
        """
        This function is as same as the dfs_helper function, but first check if the given word is in trie or not, then
        call the the dfs_helper with the root node and return all words contain x as prefix(include x if there exist).
        :parameter
            - x:str
                the given word or prefix
        :return
            return all words contain x as prefix(include x if there exist).
        """
        node = self.root
        for char in x:
            if char in node.children:
                node = node.children[char]
            else:
                return []
        result = self.dfs_helper(node, x[:-1])
        return sorted(result)

## querying
At last we complete our IR system by implementing a QueryProcessor class. This class will use the InvertedIndex class we defined before.

In [6]:
class QueryProcessor:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    indexing_model: InvertedIndex
        an instance of InvertedIndex class which has been created by indexing documents.
    Methods
    -------
    Methods defined here:
        get_word_docs(self, word: str):
                this simple function gets a token and will return all index of documents that this token is appeared in.
                :param word:
                    a word that you want to search.
                :return:
                    list of indexes of documents.
        intersect(self, first_word, second_word):
                this function get two words, and find documents that both of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            union(self, first_word, second_word):
                this function get two words, and find documents that either each of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            not_in(self, word):
                this function get one word, and find documents that this word has been not occurred.
                :parameter
                word: str
                    the word you want to search
                :return
                    list of indexes of documents.
            near(self, first_word, second_word, length):
                this function get two words, and find documents that either each of these word has been occurred near by at most 3 words on left or right.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            search(self, query):
                this function get a query and recognize what kind of query is; then search the query.
                :parameter
                    query: str
                        the query that user wants to search
                :return
                    print list of indexes of documents in a pretty way.
    """

    def __init__(self, indexing_model):
        self.indexing_model = indexing_model
        self.prefix_trie: Trie = Trie()

    def get_word_docs(self, word):
        t = self.indexing_model.get_token(word)
        result = set()
        for doc in t.docs:
            result.add(doc['doc_idx'])
        return result

    def create_prefix_trie(self):
        """
        This functions should run after create_posting_list to add all words to prefix_trie to create it.
        :return:
            None
        """
        if self.indexing_model.posting_list:
            for token in self.indexing_model.posting_list:
                # Add permuterms
                permuterms = get_all_permutations(token.word + '$')
                for term in permuterms:
                    node = self.prefix_trie.insert(term)
        else:
            raise Exception("You should first create posting list")

    def wildcard_query(self, token: str):
        """
        This function gets a token that contain * and return matched word.
        If there is just one *, it will add a '$' at the end of the word, and then permutate it such that the * pose on
        the end of the string, and then search it in prefix_trie and will return the all match tokens.
        If there are two *, it convert two * and the characters between these two into one *. Then do the same thing for
        having one *. And from resulted matched, it search if they have the sequence characters of between two * in the
        input token.
        :param token:
            the token that has * and you wants to get all matches.
        :return:
            return all matches
        """
        if token.count('*') == 1:
            star_idx = token.index('*')
            result_tokens = self.prefix_trie.query(token[star_idx + 1:] + token[0:star_idx])
            for i, word in enumerate(result_tokens):
                dollar_idx = word.index('$')
                result_tokens[i] = word[dollar_idx + 1:] + word[0:dollar_idx]
            return result_tokens

        elif token.count('*') == 2:
            star1_idx = token.index('*')
            star2_idx = token.index('*', star1_idx + 1)
            result_tokens = self.prefix_trie.query(token[star2_idx + 1:] + token[0:star1_idx])
            for i, word in enumerate(result_tokens):
                dollar_idx = word.index('$')
                result_tokens[i] = word[dollar_idx + 1:] + word[0:dollar_idx]
            # print(result_tokens)
            result = []
            for word in result_tokens:
                if token[star1_idx + 1:star2_idx] in word:
                    result.append(word)
            return result
        else:
            raise Exception("Query is not valid")

    def intersect(self, first_word, second_word):
        docs1 = self.get_word_docs(first_word)
        docs2 = self.get_word_docs(second_word)
        return set(docs1 & docs2)

    def union(self, first_word, second_word):
        docs1 = self.get_word_docs(first_word)
        docs2 = self.get_word_docs(second_word)
        return set(docs1 | docs2)

    def not_in(self, word):
        all_docs = set(range(len(self.indexing_model.documents)))
        word_docs = self.get_word_docs(word)
        return set(all_docs - word_docs)

    def near(self, first_word, second_word, distance):
        result = set()
        t1 = self.indexing_model.get_token(first_word)
        t2 = self.indexing_model.get_token(second_word)
        for t in (t1, t2):
            for doc in t.docs:
                for idx in doc['indexes']:
                    if second_word in self.indexing_model.documents[doc["doc_idx"]][idx + 1:idx + 1 + distance]:
                        result.add(doc['doc_idx'])
        return set(result)

    def search(self, query):
        query_parts = query.lower().split()
        if len(query_parts) > 2:
            if '*' not in query_parts[0]:
                query_parts[0] = [query_parts[0]]
            if '*' not in query_parts[2]:
                query_parts[2] = [query_parts[2]]
            if '*' in query_parts[0]:
                query_parts[0] = self.wildcard_query(query_parts[0] + '$')
            if '*' in query_parts[2]:
                query_parts[2] = self.wildcard_query(query_parts[1] + '$')
        if len(query_parts) == 1:
            if '*' in query_parts[0]:
                query_parts[0] = self.wildcard_query(query_parts[0] + '$')
            else:
                query_parts[0] = [query_parts[0]]
            result = self.get_word_docs(query_parts[0][0])
            for token1 in query_parts[0]:
                result = result.union(self.get_word_docs(token1))
            return result
        elif 'and' in query_parts:
            result = set()
            for token1 in query_parts[0]:
                for token2 in query_parts[2]:
                    result = result.union(self.intersect(token1, token2))
            return result
        elif 'or' in query_parts:
            result = set()
            for token1 in query_parts[0]:
                for token2 in query_parts[2]:
                    result = result.union(self.union(token1, token2))
            return result
        elif 'not' in query_parts:
            if '*' in query_parts[0]:
                query_parts[1] = self.wildcard_query(query_parts[1] + '$')
            else:
                query_parts[1] = [query_parts[1]]
            result = self.not_in(query_parts[1][0])
            for token1 in query_parts[1]:
                result = result.intersection(self.not_in(token1))
            return result
        elif 'near' in query:
            distance = int(query_parts[1].split('/')[1])
            result = set()
            for token1 in query_parts[0]:
                for token2 in query_parts[2]:
                    result = result.union(self.near(token1, token2, distance=distance))
            return self.near(query_parts[0], query_parts[2], distance)
        else:
            return list(self.get_word_docs(query_parts[0]))


# Test
Now let's test our system using samples

### read documents
First let's define a documetns_test list.

In [7]:
# Sample documents:
documents = [
    'This is a simple example document. It contains several words. The words should be processed and indexed.',
    'Another example document with different content. Document indexing is important for retrieval.',
    'Another example document to test Boolean search capabilities. This document contains relevant content.',
    'Automat Automata Automation Automatic nobody nood need nid nobody nearby nekoray neyshabour nooobbbbboooy']

### Build our system

# Create an object of our system
ir_system = InformationRetrievalSystem(documents_test)
# Tokenize documents
ir_system.tokenize(use_nltk=True)
# Create posting_list
ir_system.create_posting_list()
# Create the prefix_trie
ir_system.create_prefix_trie()Now let's build our system and call initial functions:

In [8]:
# Preprocess documents
preprocessed_documents = preprocess_documents(documents)

# Initialize the Information Retrieval System with the preprocessed documents
inverted_index = InvertedIndex(preprocessed_documents)

In [9]:
# Create posting lists
inverted_index.create_posting_list()

# Let's see the posting list
inverted_index.posting_list


[another,
 automat,
 automata,
 automatic,
 automation,
 boolean,
 capabilities,
 contains,
 content,
 different,
 document,
 example,
 important,
 indexed,
 indexing,
 nearby,
 need,
 nekoray,
 neyshabour,
 nid,
 nobody,
 nood,
 nooobbbbboooy,
 processed,
 relevant,
 retrieval,
 search,
 several,
 simple,
 test,
 words]

In [10]:
print('**** ir_system documents:\n', inverted_index.documents)

**** ir_system documents:
 [['simple', 'example', 'document', 'contains', 'several', 'words', 'words', 'processed', 'indexed'], ['another', 'example', 'document', 'different', 'content', 'document', 'indexing', 'important', 'retrieval'], ['another', 'example', 'document', 'test', 'boolean', 'search', 'capabilities', 'document', 'contains', 'relevant', 'content'], ['automat', 'automata', 'automation', 'automatic', 'nobody', 'nood', 'need', 'nid', 'nobody', 'nearby', 'nekoray', 'neyshabour', 'nooobbbbboooy']]


In [11]:
print('**** ir_system posting list:\n', inverted_index.posting_list)

**** ir_system posting list:
 [another, automat, automata, automatic, automation, boolean, capabilities, contains, content, different, document, example, important, indexed, indexing, nearby, need, nekoray, neyshabour, nid, nobody, nood, nooobbbbboooy, processed, relevant, retrieval, search, several, simple, test, words]


### Testing
Now we can test and query to our system. Note that the documents are 0-based indexing:

In [12]:
ir_system = QueryProcessor(inverted_index)
ir_system.create_prefix_trie()

In [13]:
# Execute a standard Boolean query
print(ir_system.search('example and content'))

# Execute a proximity query
print(ir_system.search('not example'))

# Execute an OR query
print(ir_system.search('example or content'))

# Execute a NOT query
print(ir_system.search('not example'))

# Execute wildcard queries and spell correction
print(ir_system.search('not t*'))
print(ir_system.search('exa*le and contrnt'))
print(ir_system.search('n*d and Automation'))
print(ir_system.search('n*b*y and Automation'))

{1, 2}
{3}
{0, 1, 2}
{3}
{0, 1, 2}
{1, 2}
{3}
{3}


### Let's test it on our dataset file too. NOTE that the documents should be provided in the same directory:

In [14]:
dataset_path = '../dataset/raw'
documents_test = []
for filename in os.listdir(dataset_path):
    filepath = os.path.join(dataset_path, filename)
    if filepath.endswith('.txt'):
        print(filename)
        # Use encoding cp1252 for test cases if an error raised
        documents_test.append(open(filepath, encoding='cp1252').read())

Happy and Unhappy Renters.txt
A Festival of Books.txt
A Murder-Suicide.txt
Jerry Decided To Buy a Gun.txt
Better To Be Unlucky.txt
Pulling Out Nine Tons of Trash.txt
Food Fight Erupted in Prison.txt
Crazy Housing Prices.txt
Trees Are a Threat.txt
Man Injured at Fast Food Place.txt
Freeway Chase Ends at Newsstand.txt
Cloning Pets.txt
Sara Went Shopping.txt
Gasoline Prices Hit Record High.txt
Rentals at the Oceanside Community.txt


### Build our system
Now let's build our system and call initial functions:

In [15]:
# Preprocess documents
preprocessed_documents = preprocess_documents(documents_test)

# Initialize the Information Retrieval System with the preprocessed documents
inverted_index = InvertedIndex(preprocessed_documents)

In [16]:
# Create posting lists
inverted_index.create_posting_list()

# Let's see the posting list
inverted_index.posting_list

[1,
 10,
 1000,
 1000x,
 100x,
 10x,
 11,
 110000,
 12,
 120,
 120000,
 15,
 150,
 15percent,
 1992,
 1993,
 1995,
 1x,
 20,
 200,
 2000,
 20000,
 209,
 214,
 222,
 230000,
 24yearold,
 25,
 25000,
 25000we,
 280,
 2995,
 3,
 30,
 300,
 3037,
 350,
 38,
 3995,
 4000,
 4000residents,
 5,
 50,
 500,
 50000,
 500x,
 50th,
 50x,
 510000it,
 60,
 6000,
 65yearold,
 7,
 70000,
 70yearold,
 74yearold,
 75,
 75000,
 76,
 79yearold,
 87,
 9,
 90,
 900,
 9000,
 99,
 able,
 abuse,
 accidentally,
 according,
 acres,
 actually,
 administration,
 afghan,
 agency,
 ago,
 aid,
 alan,
 allen,
 allotments,
 allow,
 allowing,
 almost,
 alone,
 along,
 already,
 also,
 altadena,
 although,
 always,
 ambulance,
 american,
 ammunition,
 among,
 amount,
 amounts,
 amputated,
 angelenos,
 angeles,
 anniversary,
 annual,
 another,
 antennas,
 anyone,
 anything,
 apartment,
 apartments,
 apparent,
 appears,
 appliances,
 apply,
 approved,
 april,
 arcadia,
 area,
 arizona,
 around,
 arrived,
 asked,
 asking,
 a

In [17]:
print('**** ir_system documents:\n', inverted_index.documents)

**** ir_system documents:
 [['samantha', 'like', 'many', 'renters', 'tired', 'renting', 'one', 'reason', 'annual', 'rent', 'goes', 'like', 'clockwork', 'every', 'year', 'landlord', 'raises', 'rent', 'five', 'percent', 'another', 'reason', 'neighbors', 'new', 'neighbors', 'always', 'seem', 'inconsiderate', 'ones', 'moved', 'said', 'first', 'neighbor', 'doorslammer', 'always', 'knew', 'came', 'home', 'left', 'home', 'moved', 'saxophonist', 'moved', 'saxophonist', 'practiced', 'two', 'hours', 'day', 'saturday', 'friends', 'would', 'come', 'id', 'get', 'listen', 'whole', 'band', 'called', 'police', 'said', 'saxophone', 'playing', 'permitted', 'apartments', 'four', 'hours', 'day', 'saxophone', 'playing', 'jobrelated', 'told', 'lucky', 'guy', 'playing', 'two', 'hours', 'daythere', 'many', 'unhappy', 'renters', 'also', 'happy', 'renters', 'ive', 'lucky', 'whole', 'life', 'said', 'howard', 'middleaged', 'man', 'neighbors', 'couldnt', 'better', 'picked', 'one', 'neighbor', 'chef', 'hed', 'bring

In [18]:
print('**** ir_system documents:\n', inverted_index.posting_list)

**** ir_system documents:
 [1, 10, 1000, 1000x, 100x, 10x, 11, 110000, 12, 120, 120000, 15, 150, 15percent, 1992, 1993, 1995, 1x, 20, 200, 2000, 20000, 209, 214, 222, 230000, 24yearold, 25, 25000, 25000we, 280, 2995, 3, 30, 300, 3037, 350, 38, 3995, 4000, 4000residents, 5, 50, 500, 50000, 500x, 50th, 50x, 510000it, 60, 6000, 65yearold, 7, 70000, 70yearold, 74yearold, 75, 75000, 76, 79yearold, 87, 9, 90, 900, 9000, 99, able, abuse, accidentally, according, acres, actually, administration, afghan, agency, ago, aid, alan, allen, allotments, allow, allowing, almost, alone, along, already, also, altadena, although, always, ambulance, american, ammunition, among, amount, amounts, amputated, angelenos, angeles, anniversary, annual, another, antennas, anyone, anything, apartment, apartments, apparent, appears, appliances, apply, approved, april, arcadia, area, arizona, around, arrived, asked, asking, assisted, attendance, attitudes, attract, audience, authors, auto, autograph, availablethis, a

In [19]:
ir_system = QueryProcessor(inverted_index)
ir_system.create_prefix_trie()
# Now we test our sample
print(ir_system.search('buy and car'))
print(ir_system.search('fire or water'))
print(ir_system.search('not example'))
print(ir_system.search('s*a and house'))
print(ir_system.search('s*a or house'))

{12, 5}
{8, 10, 5}
{6, 7, 8, 9, 10, 11, 12, 13, 14}
{7}
{0, 2, 7, 12, 14}
